In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
import json
import requests

In [2]:
def register_plate(license_plate: str, parker_name: str, parker_number: str) -> str:
    """Register a car with parking service"""
    config_id = int(os.getenv("CONFIG_ID", "").strip())
    location_id = int(os.getenv("LOCATION_ID", "").strip())
    tenant_id = int(os.getenv("TENANT_ID", "").strip())
    state = os.getenv("STATE", "").strip()
    parking_code = os.getenv("PARKING_CODE", "").strip()
    parker_name_id = int(os.getenv("PARKER_NAME_ID", "").strip())
    parker_contact_number_id = int(os.getenv("PARKER_CONTACT_NUMBER_ID", "").strip())
    contact_email = os.getenv("CONTACT_EMAIL", "").strip()
    url = "https://ogr-api.offstreet.io/v2/portal/registration"
    payload = json.dumps({
        "configId": config_id,
        "locationId": location_id,
        "tenantId": tenant_id,
        "vehicle": {
            "plate": license_plate,
            "state": state
        },
        "code": parking_code,
        "timeRaw": {
            "start": "2024-03-11T22:47:00.000Z",
            "end": "2024-03-12T22:47:00.000Z",
            "durationInMinutes": 1440
        },
        "additionalFieldValues": [
            {
                "id": parker_name_id,
                "label": "Parker Name",
                "value": parker_name,
                "additionalField": {
                    "id": parker_name_id
                }
            },
            {
                "id": parker_contact_number_id,
                "label": "Contact Number",
                "value": parker_number,
                "additionalField": {
                    "id": parker_contact_number_id
                }
            }
        ],
        "ignoreExistingRegistration": True
    })
    headers = {
        'Content-Type': 'application/json'
    }
    response = requests.post(url, headers=headers, data=payload)
    response_dict = json.loads(response.text)
    confirmation_number = response_dict.get('confirmation')
    if confirmation_number:
        conf_url = f"https://ogr-api.offstreet.io/v2/portal/registration/{confirmation_number}/sendParkingConfirmationEmail"
        payload2 = json.dumps({"recipient": contact_email})
        requests.post(conf_url, headers=headers, data=payload2)
        return "Plate Registered"
    else:
        print(f"Error: {response_dict}")
        return "Registration failed"

In [3]:
register_plate("LMN456", "Alex Johnson", "555-3456")

'Plate Registered'